Hello everyone,
I'm Prof.Pushplata Dubey, and I'm excited to introduce one of the key tools I've developed to help summarize the videos efficiently.
I hope you find it useful and insightful!

Follow these steps to get your summarized videos !

Extract Sentences from Paragraph and Save to CSV

In [ ]:
import spacy
import pandas as pd

def paragraph_to_sentences_csv(paragraph, output_csv="sentences.csv"):

    nlp = spacy.load("en_core_web_sm")
    doc = nlp(paragraph)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    df = pd.DataFrame(sentences, columns=["sentence"])
    df.to_csv(output_csv, index=False)
    print(f"Saved {len(sentences)} sentences to '{output_csv}'")
    return df
if __name__ == "__main__":
    paragraph = """
    Audio can often be more beneficial than visual information in video summarization.
    Especially when spoken content or sound cues are central to understanding the context.
    Prior research has used speech transcripts to generate summaries.
    """
    df = paragraph_to_sentences_csv(paragraph)
    print(df)

Saved 3 sentences to 'sentences.csv'
                                            sentence
0  Audio can often be more beneficial than visual...
1  Especially when spoken content or sound cues a...
2  Prior research has used speech transcripts to ...


Enter your filename or load it from your Google-drive

In [ ]:
filename = "/content/drive/MyDrive/ve/sports.mp4" #Enter your filename here

In [ ]:
!pip install -q openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/

Install and Load Whisper Model

In [ ]:
from google.colab import files
import whisper
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 82.6MiB/s]


ranscribe Video Audio and Save Transcript

In [ ]:
import whisper
result = model.transcribe(filename, verbose=False)
print(result['text'])

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: English


 96%|█████████▋| 632349/656349 [22:58<00:52, 458.70frames/s]

 Manchester United's goalkeeper, Talis Joyce, looking at the picture of a focus person. The Latissier United captain, such a consistent player, for United as is, Alessons captain Kim Little. He'll be leading her team out to Champions League final. The league's top goal scorers are also versus the league's best defense. Here we go then, the final day of the WSL season. It's second versus third. The league's top goalkeeper, Talis Joyce, is the top goalkeeper, for United as is. The league's side, second place, for the defeat, would see Manchester United leapfrog them. Arsenal come into today on the back of two heavy league defeats. The team is 5-2 lost to Astrid and Villa more understandable as it was a rotated side but Monday's lost to Brighton. The league's goalkeeper, Chloe Kelly, preferred to Beth Mead, a landmark appearance for Fruda Moorner who makes her 150th Arsenal appearance. Today she's been a consistent player in this recent Arsenal team only missing two WSL games in the last 

In [ ]:
import pandas as pd
df = pd.DataFrame(result['segments'])
df = df[['start', 'end', 'text']]
df.to_csv("transcript.csv", index=False)#Save in transcript.csv

Merge Transcript Segments into Complete Sentences with Timestamps

In [ ]:
import pandas as pd
import re
df = pd.read_csv("transcript.csv")
df['text'] = df['text'].fillna('').astype(str)
merged_data = []
buffer_text = ""
timestamps = []

for i in range(len(df)):
    current = df.iloc[i]
    text = current['text'].strip()

    if not text:
        continue

    if not buffer_text:
        buffer_text = text
        timestamps = [(current['start'], current['end'])]
    else:
        buffer_text += " " + text
        timestamps.append((current['start'], current['end']))


    is_end_of_sentence = text[-1] in ".!?"
    next_is_continuation = False
    if i + 1 < len(df):
        next_text = df.iloc[i + 1]['text'].strip()
        next_is_continuation = (
            next_text and (
                next_text[0].islower() or
                next_text[0] in "\"'(-" or
                len(next_text.split()) < 4
            )
        )

    if is_end_of_sentence or not next_is_continuation or i == len(df) - 1:
        buffer_text = buffer_text.strip()
        if buffer_text and buffer_text[-1] not in ".!?":
            buffer_text += "."
        sentences = re.split(r'(?<=[.!?])\s+', buffer_text)
        for ts in timestamps:
            if ts[1] > ts[0]:
                sentence_start = ts[0]
                break
        total_words = sum(len(s.split()) for s in sentences)
        total_duration = timestamps[-1][1] - timestamps[0][0]
        time_per_word = total_duration / total_words if total_words else 0

        word_count = 0
        for sent in sentences:
            if not sent.strip():
                continue
            sent_words = len(sent.split())
            duration = sent_words * time_per_word
            sentence_end = sentence_start + duration
            merged_data.append({
                "start": round(sentence_start, 2),
                "end": round(sentence_end, 2),
                "text": sent.strip(),
                "score": 0  # ADDING score column with default 0
            })
            sentence_start = sentence_end

        buffer_text = ""
        timestamps = []
final_df = pd.DataFrame(merged_data)
final_df.to_csv("corrected_transcript.csv", index=False)



Select Top Sentences for Summary Using TF-IDF Scoring

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from moviepy.editor import VideoFileClip


df = pd.read_csv("corrected_transcript.csv")
video = VideoFileClip(filename)
video_duration = video.duration # in seconds

#Target summary duration(50% of video)
target_summary_duration = video_duration * 0.5
#Calculate average sentence duration
df['duration'] = df['end'] - df['start']
average_sentence_duration = df['duration'].mean()
#How many top sentences needed
num_sentences = int(target_summary_duration / average_sentence_duration)
num_sentences = min(num_sentences, len(df))  # In case sentences are fewer
print(f"Video Duration: {video_duration:.2f}s, Target Summary: {target_summary_duration:.2f}s")
print(f"Average Sentence Duration: {average_sentence_duration:.2f}s")
print(f"Selecting Top {num_sentences} sentences")

#TF-IDF based scoring
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
scores = np.asarray(X.sum(axis=1)).ravel()

#Update score column in df
df['score'] = scores
df.to_csv("scored_corrected_transcript.csv", index=False)
top_indices = scores.argsort()[-num_sentences:][::-1]# highest scores first
top_sentences = df.iloc[top_indices]
top_sentences = top_sentences.sort_values(by="start").reset_index(drop=True)
top_sentences.to_csv("top_sentences.csv", index=False)
with open("corrected_transcript.txt", "w") as f:
    for line in df['text']:
        f.write(line + "\n")


  if event.key is 'enter':



Video Duration: 6563.49s, Target Summary: 3281.74s
Average Sentence Duration: 5.40s
Selecting Top 608 sentences


Generate Highlight Video Clips from Top Sentences

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import pandas as pd
import os
import gc
top_sentences = pd.read_csv("top_sentences.csv")
video = VideoFileClip(filename)
original_duration = video.duration
target_duration = original_duration * 0.5
print(f"Original video duration: {original_duration/60:.2f} mins")
print(f"Target highlight video duration: {target_duration/60:.2f} mins")

top_sentences['duration'] = top_sentences['end'] - top_sentences['start']

top_sentences = top_sentences.sort_values(by="score", ascending=False).reset_index(drop=True)
total_top_duration = top_sentences['duration'].sum()
while total_top_duration > target_duration and len(top_sentences) > 1:
    removed_row = top_sentences.iloc[-1]
    print(f"Removing sentence starting at {removed_row['start']}s with score {removed_row['score']:.2f} to fit target duration.")
    top_sentences = top_sentences.iloc[:-1]
    total_top_duration = top_sentences['duration'].sum()
top_sentences.to_csv("selected_top_sentences_by_score.csv", index=False)
top_sentences = top_sentences.sort_values(by="start").reset_index(drop=True)

print(f"Final number of clips: {len(top_sentences)}, Final highlight duration: {total_top_duration/60:.2f} mins")
valid_clips = []
os.makedirs("clips", exist_ok=True)
epsilon = 0.05  # small cut to avoid MoviePy warning

for idx, row in top_sentences.iterrows():
    start = row['start']
    end = row['end']
    safe_end = max(start, end - epsilon)

    if (safe_end - start) < 0.1:
        print(f"Skipping too short clip: {start} to {end}")
        continue

    clip = video.subclip(start, safe_end)
    path = f"clips/clip_{idx}.mp4"
    clip.write_videofile(path, codec="libx264", audio_codec="aac", verbose=False, logger=None)
    valid_clips.append(path)

video.close()
gc.collect()



Original video duration: 109.39 mins
Target highlight video duration: 54.70 mins
Removing sentence starting at 3295.46s with score 2.17 to fit target duration.
Removing sentence starting at 5944.22s with score 2.17 to fit target duration.
Removing sentence starting at 3831.5s with score 2.17 to fit target duration.
Removing sentence starting at 4461.72s with score 2.17 to fit target duration.
Removing sentence starting at 316.0s with score 2.17 to fit target duration.
Removing sentence starting at 1416.56s with score 2.17 to fit target duration.
Removing sentence starting at 5659.22s with score 2.17 to fit target duration.
Removing sentence starting at 5426.72s with score 2.17 to fit target duration.
Removing sentence starting at 5155.72s with score 2.17 to fit target duration.
Removing sentence starting at 3380.46s with score 2.17 to fit target duration.
Removing sentence starting at 6090.22s with score 2.17 to fit target duration.
Removing sentence starting at 1486.3s with score 2.18

24

Extract Keyframes Using CNN-Based Feature Similarity resnet 50

In [ ]:
# Install required libraries
!pip install opencv-python-headless torch torchvision pandas

# Imports
import os
import cv2
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from google.colab import files
from torch.nn.functional import normalize

# Load Pretrained ResNet50 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove final classification layer
model = model.to(device)
model.eval()

# Transformation for frames
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Feature extraction function
def get_features(frame):
    frame = transform(frame).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(frame)
    features = features.view(-1)  # Flatten
    features = normalize(features.unsqueeze(0)).squeeze(0)  # L2 normalize
    return features.cpu().numpy()

# Main function for keyframe extraction + CSV saving
def extract_keyframes_cnn(video_path, output_folder, csv_filename="keyframes_with_timestamps.csv", feature_threshold=0.8):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    success, prev_frame = cap.read()
    frame_count = 0
    keyframe_id = 0  # Sequential id: 0,1,2,...
    saved_count = 0

    if not success:
        print("Error reading video.")
        return

    prev_features = get_features(prev_frame)
    keyframe_data = []  # List to store info: frame_id, filename, timestamp

    while True:
        success, frame = cap.read()
        if not success:
            break

        current_features = get_features(frame)
        similarity = np.dot(prev_features, current_features)

        if similarity < feature_threshold:
            frame_filename = f"keyframe_{frame_count:05d}.jpg"
            frame_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(frame_path, frame)
            saved_count += 1

            # Save sequential frame_id, filename and timestamp
            timestamp = frame_count / fps  # seconds
            keyframe_data.append({
                "frame_id": keyframe_id,  # sequential 0,1,2,...
                "filename": frame_filename,
                "timestamp_sec": round(timestamp, 2)
            })
            keyframe_id += 1  # increment sequential id

            prev_features = current_features

        frame_count += 1

    cap.release()

    # Save keyframe data to CSV
    df = pd.DataFrame(keyframe_data)
    df.to_csv(os.path.join(output_folder, csv_filename), index=False)

    print(f"✅ Extraction completed! {saved_count} key frames saved at {output_folder}")
    print(f"✅ CSV file saved: {csv_filename}")

# Upload your video
#uploaded = files.upload()

video_filename = filename
output_folder = "/content/keyframes_resnet50"
csv_filename = "keyframes_with_timestamps.csv"

# Extract keyframes + Save CSV
extract_keyframes_cnn(video_filename, output_folder, csv_filename=csv_filename, feature_threshold=0.8)

# Optional: Zip and Download output
!zip -r keyframes_resnet50.zip /content/keyframes_resnet50
files.download('keyframes_resnet50.zip')


  warnings.warn(

  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 210MB/s]


✅ Extraction completed! 1563 key frames saved at /content/keyframes_resnet50
✅ CSV file saved: keyframes_with_timestamps.csv
  adding: content/keyframes_resnet50/ (stored 0%)
  adding: content/keyframes_resnet50/keyframe_102359.jpg (deflated 2%)
  adding: content/keyframes_resnet50/keyframe_13032.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_133167.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_147038.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_38284.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_31247.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_56035.jpg (deflated 1%)
  adding: content/keyframes_resnet50/keyframe_04722.jpg (deflated 1%)
  adding: content/keyframes_resnet50/keyframe_24916.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_148648.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_143952.jpg (deflated 0%)
  adding: content/keyframes_resnet50/keyframe_1402

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Refine Keyframes Using Scene Change Detection via Histogram Comparison

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np

def refine_keyframes_with_scene_change(keyframes_csv, video_path, refined_csv_path, hist_threshold=0.6):
    keyframes_df = pd.read_csv(keyframes_csv)
    cap = cv2.VideoCapture(video_path)

    refined_keyframes = []
    last_hist = None

    for _, row in keyframes_df.iterrows():
        timestamp = row['timestamp_sec']
        cap.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
        success, frame = cap.read()
        if not success:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()

        if last_hist is None:
            refined_keyframes.append(row)
            last_hist = hist
        else:
            diff = cv2.compareHist(last_hist, hist, cv2.HISTCMP_BHATTACHARYYA)
            if diff > hist_threshold:
                refined_keyframes.append(row)
                last_hist = hist

    cap.release()

    pd.DataFrame(refined_keyframes).to_csv(refined_csv_path, index=False)
    print(f"🎬 Scene-based refinement done! {len(refined_keyframes)} frames kept.")
    print(f"📁 Saved to: {refined_csv_path}")


In [ ]:
refine_keyframes_with_scene_change(
    keyframes_csv="/content/keyframes_resnet50/keyframes_with_timestamps.csv",
    video_path=video_filename,
    refined_csv_path="/content/keyframes_resnet50/refined_keyframes_scene.csv",
    hist_threshold=0.6  # You can tune this: lower = stricter change detection
)

🎬 Scene-based refinement done! 237 frames kept.
📁 Saved to: /content/keyframes_resnet50/refined_keyframes_scene.csv


Save and Download Refined Keyframes as Images

In [ ]:
import cv2
import pandas as pd
import os
import zipfile
from google.colab import files

# Paths
refined_csv = "/content/keyframes_resnet50/refined_keyframes_scene.csv"
video_path = filename
output_folder = "/content/refined_frames"
zip_path = "/content/refined_keyframes.zip"

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Load refined timestamps
refined_df = pd.read_csv(refined_csv)
cap = cv2.VideoCapture(video_path)

# Extract and save frames
for idx, row in refined_df.iterrows():
    timestamp = row['timestamp_sec']
    cap.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
    success, frame = cap.read()
    if success:
        frame_path = os.path.join(output_folder, f"frame_{idx:03d}_{int(timestamp)}s.jpg")
        cv2.imwrite(frame_path, frame)

cap.release()

# Zip the folder
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, _, files_in_dir in os.walk(output_folder):
        for file in files_in_dir:
            zipf.write(os.path.join(root, file), arcname=file)

# Download the zip
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Final Transcript Completion: Merging Highlights with Scene-Based Keyframes

In [ ]:
import pandas as pd

# Load the files
corrected_transcript = pd.read_csv('corrected_transcript.csv')
selected_sentences = pd.read_csv('/content/selected_top_sentences_by_score.csv')
keyframes = pd.read_csv('/content/keyframes_resnet50/refined_keyframes_scene.csv')

# Convert to float for safety
for col in ['start', 'end']:
    corrected_transcript[col] = corrected_transcript[col].astype(float)
    selected_sentences[col] = selected_sentences[col].astype(float)
keyframes['timestamp_sec'] = keyframes['timestamp_sec'].astype(float)

# Sort everything
corrected_transcript = corrected_transcript.sort_values(by='start').reset_index(drop=True)
selected_sentences = selected_sentences.sort_values(by='start').reset_index(drop=True)

# Prepare buffer and tracking
final_transcript_buffer = []
existing_texts = set()
missing_frames_counter = 0

# Helper function to find nearest original transcript for a timestamp
def find_original_text(ts):
    for idx, row in corrected_transcript.iterrows():
        if row['start'] <= ts <= row['end']:
            return row
    return None  # If no matching found (rare case)

# Go through keyframes
for idx, row in keyframes.iterrows():
    frame_timestamp = row['timestamp_sec']

    # Check if frame timestamp falls inside any selected sentence range
    found_in_selected = False
    for _, sent_row in selected_sentences.iterrows():
        if sent_row['start'] <= frame_timestamp <= sent_row['end']:
            found_in_selected = True
            break

    if found_in_selected:
        continue  # Already covered, skip to next frame

    # Not found: find from corrected transcript
    nearest_text_row = find_original_text(frame_timestamp)
    if nearest_text_row is not None:
        text_to_add = nearest_text_row['text'].strip()

        if text_to_add not in existing_texts:
            final_transcript_buffer.append({
                "start": nearest_text_row['start'],
                "end": nearest_text_row['end'],
                "text": text_to_add
            })
            existing_texts.add(text_to_add)
            missing_frames_counter += 1

# Now add all selected sentences to buffer too
for _, sent_row in selected_sentences.iterrows():
    text_to_add = sent_row['text'].strip()
    if text_to_add not in existing_texts:
        final_transcript_buffer.append({
            "start": sent_row['start'],
            "end": sent_row['end'],
            "text": text_to_add
        })
        existing_texts.add(text_to_add)

# Sort the final buffer by start time
final_transcript_buffer = sorted(final_transcript_buffer, key=lambda x: x['start'])

# Save final transcript
final_df = pd.DataFrame(final_transcript_buffer)
final_df.to_csv('final_transcript_with_missing_added.csv', index=False)

print(f"✅ Done! Final transcript saved.")
print(f"ℹ️ Missing frames handled and added from original transcript: {missing_frames_counter} frames.")


✅ Done! Final transcript saved.
ℹ️ Missing frames handled and added from original transcript: 63 frames.


Generate Highlight Clips from Final Transcript

In [ ]:
import moviepy.editor as mp
import pandas as pd
import os
import gc

# Load transcript
final_transcript = pd.read_csv('final_transcript_with_missing_added.csv', usecols=['start', 'end'])

# Load video
#filename = "/content/drive/MyDrive/ve/sports.mp4" # Insert your filename
video_path = filename  # Define this if not already
full_video = mp.VideoFileClip(video_path)

# Create output folder
clip_output_folder = '/content/temp_clips'
os.makedirs(clip_output_folder, exist_ok=True)

clip_paths = []

# Step 1: Save each clip to file to reduce RAM usage
for idx, row in final_transcript.iterrows():
    start = float(row['start'])
    end = float(row['end'])

    # Avoid empty/invalid clips
    if end - start < 0.1:
        continue

    subclip = full_video.subclip(start, end)
    path = os.path.join(clip_output_folder, f'clip_{idx}.mp4')
    subclip.write_videofile(path, codec='libx264', audio_codec='aac', verbose=False, logger=None)
    clip_paths.append(path)

    # Free memory after each clip
    del subclip
    gc.collect()

# Release full video
full_video.close()
del full_video
gc.collect()



5

 Merge Clips into Final Summary Video

In [ ]:
import moviepy.editor as mp
import os
import gc
import shutil
import pandas as pd
from google.colab import files

# Settings
clip_dir = '/content/temp_clips'
csv_path = '/content/final_transcript_with_missing_added.csv'
chunk_size = 50
intermediate_dir = '/content/intermediate_clips'
final_video_path = '/content/final_summary_video.mp4'

# Load CSV with timestamps
df = pd.read_csv(csv_path, usecols=['start', 'end'])

# Sort CSV by start time to ensure correct order
df_sorted = df.sort_values(by='start').reset_index()

# Generate clip file paths based on index (assumes naming like clip_0.mp4)
clip_paths = []
for idx in df_sorted['index']:
    clip_filename = f"clip_{idx}.mp4"
    clip_path = os.path.join(clip_dir, clip_filename)
    if os.path.exists(clip_path):
        clip_paths.append(clip_path)
    else:
        print(f"⚠️ Clip not found: {clip_filename}")

if not clip_paths:
    print("❌ No valid clips found!")
else:
    print(f"🔢 Total sorted clips: {len(clip_paths)}")
    os.makedirs(intermediate_dir, exist_ok=True)
    intermediate_paths = []

    # Step 1: Process in chunks
    for i in range(0, len(clip_paths), chunk_size):
        print(f"🧩 Processing chunk {i // chunk_size + 1}...")
        chunk = clip_paths[i:i + chunk_size]
        clips = []

        for p in chunk:
            try:
                clip = mp.VideoFileClip(p)
                clips.append(clip)
            except Exception as e:
                print(f"❌ Failed to load clip {p}: {e}")

        if clips:
            intermediate_path = os.path.join(intermediate_dir, f'chunk_{i // chunk_size + 1}.mp4')
            intermediate_clip = mp.concatenate_videoclips(clips, method='compose')
            intermediate_clip.write_videofile(intermediate_path, codec='libx264', audio_codec='aac', verbose=False, logger=None)
            intermediate_paths.append(intermediate_path)

            # Clean up
            for c in clips:
                c.close()
            del clips, intermediate_clip
            gc.collect()

    # Step 2: Final concatenation
    print("🔗 Combining intermediate videos...")
    final_clips = [mp.VideoFileClip(p) for p in intermediate_paths]
    final_video = mp.concatenate_videoclips(final_clips, method='compose')
    final_video.write_videofile(final_video_path, codec='libx264', audio_codec='aac')

    # Cleanup
    for c in final_clips:
        c.close()
    del final_clips, final_video
    gc.collect()

    shutil.rmtree(intermediate_dir)
    shutil.rmtree(clip_dir)

    print("✅ Final video created!")
    files.download(final_video_path)


🔢 Total sorted clips: 497
🧩 Processing chunk 1...
🧩 Processing chunk 2...
🧩 Processing chunk 3...
🧩 Processing chunk 4...
🧩 Processing chunk 5...
🧩 Processing chunk 6...
🧩 Processing chunk 7...
🧩 Processing chunk 8...
🧩 Processing chunk 9...
🧩 Processing chunk 10...


  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



🔗 Combining intermediate videos...
Moviepy - Building video /content/final_summary_video.mp4.
MoviePy - Writing audio in final_summary_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/final_summary_video.mp4



Moviepy - Done !
Moviepy - video ready /content/final_summary_video.mp4
✅ Final video created!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This is not mandatory
It is just for clearing the cache

In [ ]:
# Clear variables
%reset -f

# Clear garbage
import gc
gc.collect()

# Clear file cache
!rm -rf __pycache__ .ipynb_checkpoints


In [ ]:
#from google.colab import files

In [ ]:
#files.download('/content/keyframes_resnet50/refined_keyframes_scene.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>